In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Reconnect to the database
conn = sqlite3.connect('./ipl_database.db')
# Query the database
df_ball = pd.read_sql_query('SELECT * FROM ipl_ball_by_ball;', conn)

# Display the first few rows of the table
print(df_ball)

# Close the connection
conn.close()


             ID  innings  overs  ballnumber       batter          bowler   
0       1312200        1      0           1  YBK Jaiswal  Mohammed Shami  \
1       1312200        1      0           2  YBK Jaiswal  Mohammed Shami   
2       1312200        1      0           3   JC Buttler  Mohammed Shami   
3       1312200        1      0           4  YBK Jaiswal  Mohammed Shami   
4       1312200        1      0           5  YBK Jaiswal  Mohammed Shami   
...         ...      ...    ...         ...          ...             ...   
225949   335982        2     14           5      P Kumar        I Sharma   
225950   335982        2     14           6     SB Joshi        I Sharma   
225951   335982        2     14           7      P Kumar        I Sharma   
225952   335982        2     15           1     SB Joshi       LR Shukla   
225953   335982        2     15           2     SB Joshi       LR Shukla   

        non-striker extra_type  batsman_run  extras_run  total_run   
0        JC Buttl

In [2]:
unique_batters = df_ball['batter'].unique().tolist()

print(unique_batters)

['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy', 'WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pandya', 'DA Miller', 'V Kohli', 'F du Plessis', 'RM Patidar', 'GJ Maxwell', 'MK Lomror', 'KD Karthik', 'Shahbaz Ahmed', 'PWH de Silva', 'HV Patel', 'JR Hazlewood', 'Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda', 'MP Stoinis', 'E Lewis', 'KH Pandya', 'PVD Chameera', 'PK Garg', 'Abhishek Sharma', 'RA Tripathi', 'AK Markram', 'N Pooran', 'Washington Sundar', 'R Shepherd', 'J Suchith', 'B Kumar', 'Umran Malik', 'JM Bairstow', 'S Dhawan', 'M Shahrukh Khan', 'MA Agarwal', 'LS Livingstone', 'JM Sharma', 'PN Mankad', 'PP Shaw', 'DA Warner', 'MR Marsh', 'RR Pant', 'SN Khan', 'R Powell', 'AR Patel', 'SN Thakur', 'Kuldeep Yadav', 'Ishan Kishan', 'RG Sharma', 'D Brevis', 'Tilak Varma', 'TH David', 'Ramandeep Singh', 'DR Sams', 'RD Gaikwad', 'DP Conway', 'MM Ali', 'N Jagadeesan', 'AT Rayudu', 'MS Dhoni', 'Simarjeet Singh', 'MJ Santner', 'R 

In [3]:
import sqlite3
import pandas as pd

def get_batting_stats(player_name):
    # Establish a connection to the SQLite database
    conn = sqlite3.connect('./ipl_database.db')

    # Define the SQL query with the corrected column name
    query = f'''
    SELECT
        innings,
        overs,
        SUM(CASE WHEN batter = ? THEN batsman_run ELSE 0 END) AS runs_scored,
        SUM((CASE WHEN isWicketDelivery = 1 AND player_out == ? THEN 1 ELSE 0 END)) AS Dismissals,
        COUNT(CASE WHEN (extra_type IS NULL OR extra_type = 'byes' OR extra_type = 'legbyes' OR extra_type = 'noballs') AND (batter = ?) THEN 1 ELSE NULL END) AS balls_faced
    FROM
        ipl_ball_by_ball
    WHERE
        batter = ? or "non-striker" = ?
    GROUP BY
        innings,
        overs
    ORDER BY
        innings,
        overs;
    '''

    # Execute the query with the player name as a parameter
    df = pd.read_sql_query(query, conn, params=(player_name, player_name, player_name, player_name, player_name))

    # Close the connection
    conn.close()

    return df

# Example usage
player_name = 'AB de Villiers'  # Replace with the actual player name
batter_stats_df = get_batting_stats(player_name)
batter_stats_df

,innings,overs,runs_scored,Dismissals,balls_faced
0,1,0,19,0,10
1,1,1,22,1,23
2,1,2,28,1,38
3,1,3,83,0,76
4,1,4,114,2,72
5,1,5,144,2,92
6,1,6,118,2,95
7,1,7,110,4,97
8,1,8,127,2,105
9,1,9,162,2,120


In [4]:
# Assuming you have a DataFrame named batter_stats_df
# Filter the DataFrame based on innings, overs range, and runs scored
innings_x = 1  # Specify the innings
overs_y = 6  # Specify the starting over
overs_z = 10  # Specify the ending over

filtered_batter_df = batter_stats_df[
    (batter_stats_df['innings'].isin([1, 2])) &  # Filter by innings
    (batter_stats_df['overs'] >= overs_y) &      # Filter by starting over
    (batter_stats_df['overs'] <= overs_z)
]

# Print the filtered DataFrame
filtered_batter_df

,innings,overs,runs_scored,Dismissals,balls_faced
6,1,6,118,2,95
7,1,7,110,4,97
8,1,8,127,2,105
9,1,9,162,2,120
10,1,10,172,5,134
26,2,6,93,5,95
27,2,7,111,3,99
28,2,8,133,4,113
29,2,9,133,3,106
30,2,10,136,4,103


In [5]:
def stat(player_name, batter_stats_df, overs_y, overs_z):
    # Assuming you have a DataFrame named batter_stats_df
    # Filter the DataFrame based on innings, overs range, and runs scored
     # Specify the ending over

    filtered_batter_df = batter_stats_df[
        (batter_stats_df['innings'].isin([1, 2])) &  # Filter by innings
        (batter_stats_df['overs'] >= overs_y) &      # Filter by starting over
        (batter_stats_df['overs'] <= overs_z)
    ]

    # Print the filtered DataFrame
    runs_score = filtered_batter_df["runs_scored"].sum()
    balls_faced = filtered_batter_df["balls_faced"].sum()
    dismissals = filtered_batter_df["Dismissals"].sum()
    average = runs_score/dismissals
    strike_rate = (runs_score/balls_faced)*100
    print(player_name)
    print(runs_score, balls_faced, dismissals, average, strike_rate)

    return [player_name, runs_score, balls_faced, dismissals, average, strike_rate]

stat("AB de Villiers", get_batting_stats("AB de Villiers"), 6, 10)

AB de Villiers
1295 1067 34 38.088235294117645 121.36832239925025


['AB de Villiers', 1295, 1067, 34, 38.088235294117645, 121.36832239925025]

In [6]:
runs_score = filtered_batter_df["runs_scored"].sum()
balls_faced = filtered_batter_df["balls_faced"].sum()
dismissals = filtered_batter_df["Dismissals"].sum()
average = runs_score/dismissals
strike_rate = (runs_score/balls_faced)*100
print(runs_score, balls_faced, dismissals, average, strike_rate)

1295 1067 34 38.088235294117645 121.36832239925025


In [7]:
def batting_stats_overs(player_name, batter_stats_df , overs_y, overs_z):

    filtered_batter_df = batter_stats_df[
        (batter_stats_df['innings'].isin([1, 2])) &  # Filter by innings
        (batter_stats_df['overs'] >= overs_y) &      # Filter by starting over
        (batter_stats_df['overs'] <= overs_z)
    ]


    runs_score = filtered_batter_df["runs_scored"].sum()
    balls_faced = filtered_batter_df["balls_faced"].sum()
    dismissals = filtered_batter_df["Dismissals"].sum()
    average = runs_score/dismissals
    strike_rate = (runs_score/balls_faced)*100
    
    return [player_name, runs_score, balls_faced, dismissals, average, strike_rate]

# Best Batters in the Powerplay

In [8]:
batting_power_play = []
for i in unique_batters:
    df = get_batting_stats(i)
    batting_power_play.append(batting_stats_overs(i, df, 0, 5))

C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: invalid value encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:14: RuntimeWarning: invalid value encountered in scalar divide
  strike_rate = (runs_score/balls_faced)*100
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  average = runs_score/dismissals


In [9]:
batting_power_play

[['YBK Jaiswal', 366, 272, 12, 30.5, 134.55882352941177],
 ['JC Buttler', 1229, 854, 26, 47.26923076923077, 143.9110070257611],
 ['SV Samson', 999, 849, 35, 28.542857142857144, 117.6678445229682],
 ['D Padikkal', 631, 512, 19, 33.21052631578947, 123.2421875],
 ['SO Hetmyer', 53, 51, 2, 26.5, 103.921568627451],
 ['R Ashwin', 58, 56, 4, 14.5, 103.57142857142858],
 ['R Parag', 0, 11, 1, 0.0, 0.0],
 ['TA Boult', 0, 0, 0, nan, nan],
 ['OC McCoy', 0, 0, 0, nan, nan],
 ['WP Saha', 972, 727, 35, 27.771428571428572, 133.70013755158183],
 ['Shubman Gill', 882, 729, 22, 40.09090909090909, 120.98765432098766],
 ['MS Wade', 113, 98, 6, 18.833333333333332, 115.3061224489796],
 ['HH Pandya', 84, 100, 3, 28.0, 84.0],
 ['DA Miller', 87, 93, 5, 17.4, 93.54838709677419],
 ['V Kohli', 2125, 1825, 58, 36.63793103448276, 116.43835616438356],
 ['F du Plessis', 1480, 1156, 38, 38.94736842105263, 128.02768166089967],
 ['RM Patidar', 107, 99, 2, 53.5, 108.08080808080808],
 ['GJ Maxwell', 349, 259, 16, 21.8125, 

In [10]:
batting_power_play_df = pd.DataFrame(batting_power_play, columns=['Batter', 'runs', 'balls', 'Dismissals', 'average', 'strike_rate'])
batting_power_play_df

,Batter,runs,balls,Dismissals,average,strike_rate
0,YBK Jaiswal,366,272,12,30.500000,134.558824
1,JC Buttler,1229,854,26,47.269231,143.911007
2,SV Samson,999,849,35,28.542857,117.667845
3,D Padikkal,631,512,19,33.210526,123.242188
4,SO Hetmyer,53,51,2,26.500000,103.921569
...,...,...,...,...,...,...
600,D Kalyankrishna,0,0,0,NaN,NaN
601,MA Khote,0,0,0,NaN,NaN
602,SB Joshi,0,0,0,NaN,NaN
603,DS Lehmann,4,3,0,inf,133.333333


In [11]:
sorted_batting_powerplay_df = batting_power_play_df.sort_values('runs', ascending=False)
filtered_sorted_batting_powerplay_df = sorted_batting_powerplay_df.query('runs >= 500').dropna()
print(len(filtered_sorted_batting_powerplay_df))
filtered_sorted_batting_powerplay_df

54


,Batter,runs,balls,Dismissals,average,strike_rate
43,S Dhawan,3157,2584,86,36.709302,122.174923
50,DA Warner,2892,2106,65,44.492308,137.321937
189,CH Gayle,2373,1761,64,37.078125,134.752981
280,G Gambhir,2277,1868,69,33.000000,121.895075
14,V Kohli,2125,1825,58,36.637931,116.438356
101,AM Rahane,2049,1783,65,31.523077,114.918676
105,RV Uthappa,2035,1636,62,32.822581,124.388753
242,PA Patel,1904,1565,73,26.082192,121.661342
59,RG Sharma,1632,1389,55,29.672727,117.494600
24,Q de Kock,1631,1248,45,36.244444,130.689103


In [12]:
# Reset index of filtered_sorted_batting_powerplay_df
filtered_sorted_batting_powerplay_df = filtered_sorted_batting_powerplay_df.reset_index(drop=True)
len(filtered_sorted_batting_powerplay_df)

54

In [60]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the data
data = filtered_sorted_batting_powerplay_df
# Handle missing values
data = data.dropna()

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(data[['runs', 'average', 'strike_rate']])

# Create and train the neural network
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(3,)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mse')
model.fit(X, X, epochs=100, batch_size=32, verbose=0)

# Inspect the model's summary to find the correct layer name
model.summary()

encoder = tf.keras.Model(model.input, model.layers[-2].output)
print(encoder)
encoded_data = encoder.predict(X)
print("encoder data")
print(encoded_data)
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(encoded_data)

# Assign the cluster labels back to the original data
data['Cluster'] = labels

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 32)                128       
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_21 (Dense)            (None, 16)                528       
                                                                 
 dropout_7 (Dropout)         (None, 16)                0         
                                                                 
 dense_22 (Dense)            (None, 8)                 136       
                                                                 
 dropout_8 (Dropout)         (None, 8)                 0         
                                                                 
 dense_23 (Dense)            (None, 1)                

c:\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [62]:
type(encoded_data)

numpy.ndarray

In [63]:
encoded_data.shape

(54, 8)

In [14]:
# Assuming 'data' is your DataFrame
sorted_data = data.sort_values(by='Cluster', ascending=False)

# Display the sorted DataFrame
sorted_data

,Batter,runs,balls,Dismissals,average,strike_rate,Cluster
0,S Dhawan,3157,2584,86,36.709302,122.174923,2
10,KL Rahul,1594,1274,28,56.928571,125.117739,2
38,JM Bairstow,695,470,14,49.642857,147.872340,2
33,CA Lynn,779,540,17,45.823529,144.259259,2
40,Ishan Kishan,679,551,12,56.583333,123.230490,2
1,DA Warner,2892,2106,65,44.492308,137.321937,2
18,JC Buttler,1229,854,26,47.269231,143.911007,2
36,SA Yadav,727,516,14,51.928571,140.891473,2
2,CH Gayle,2373,1761,64,37.078125,134.752981,2
19,DR Smith,1213,976,27,44.925926,124.282787,1


In [64]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Assuming 'data' is your DataFrame and 'X' is the standardized input features
data = filtered_sorted_batting_powerplay_df
# Handle missing values
data = data.dropna()

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(data[['runs', 'average', 'strike_rate']])
# Define the size of the encoding
encoding_dim = 2  # This is the size of our encoded representations (bottleneck)

# Define the input layer with the same number of neurons as the input features
input_data = Input(shape=(3,))

# "encoded" is the encoded representation of the input
encoded = Dense(32, activation='relu')(input_data)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)  # Bottleneck layer

# "decoded" is the lossy reconstruction of the input
decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(3, activation='linear')(decoded)  # Output layer with same size as input

# This model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

# This model maps an input to its encoded representation
encoder = Model(input_data, encoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X, X,
                epochs=100,
                batch_size=32,
                shuffle=True,
                verbose=0)

# Use the encoder to encode the data
encoded_data = encoder.predict(X)

# Apply K-Means clustering on the encoded data
kmeans = KMeans(n_clusters=3, random_state=0)
labels = kmeans.fit_predict(encoded_data)

# Assign the cluster labels back to the original DataFrame
data['Cluster'] = labels

2/2 [==============================] - 0s 2ms/step


c:\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [66]:
# Assuming 'data' is your DataFrame
sorted_data = data.sort_values(by='Cluster', ascending=False)

# Display the sorted DataFrame
sorted_data

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Cluster
41,SP Narine,665,383,36,18.472222,173.629243,0.606210,2
11,V Sehwag,1593,1105,61,26.114754,144.162896,0.493253,2
26,WP Saha,972,727,35,27.771429,133.700138,0.403628,2
22,PP Shaw,1129,766,40,28.225000,147.389034,0.530297,2
39,M Vohra,681,529,25,27.240000,128.733459,0.344091,2
32,RA Tripathi,779,549,24,32.458333,141.894353,0.526059,2
51,ST Jayasuriya,514,366,19,27.052632,140.437158,0.433574,2
18,JC Buttler,1229,854,26,47.269231,143.911007,0.752485,1
28,R Dravid,905,832,17,53.235294,108.774038,0.523028,1
45,ML Hayden,599,461,13,46.076923,129.934924,0.595999,1


In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(filtered_sorted_batting_powerplay_df[['runs', 'average', 'strike_rate']])

# Define weights for the composite score
runs_weight = 0.1
avg_weight = 0.5
strike_rate_weight = 0.6

# Create a new column for the composite score
filtered_sorted_batting_powerplay_df['performance'] = (
    runs_weight * normalized_data[:, 0] +
    avg_weight * normalized_data[:, 1] +
    strike_rate_weight * normalized_data[:, 2]
)

# Split the data into features and target variable
X = normalized_data
y = filtered_sorted_batting_powerplay_df["performance"]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100

2/2 [==============================] - 1s 5ms/step - loss: 0.6942 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6922 - accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6907 - accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6882 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6873 - accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6863 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6854 - accuracy: 0.0000e+00
Epoch 9/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6844 - accuracy: 0.0000e+00
Epoch 10/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6834 - accuracy:

In [16]:
predictions

array([[0.5042058 ],
       [0.7274901 ],
       [0.34958225],
       [0.33146742],
       [0.75575966],
       [0.27098617],
       [0.51629543],
       [0.5171015 ],
       [0.51617604],
       [0.5187535 ],
       [0.43116567]], dtype=float32)

In [17]:
len(predictions)

11

In [18]:
output_players_performance = model.predict(X)
print(output_players_performance)

2/2 [==============================] - 0s 2ms/step
[[0.5171015 ]
 [0.72802734]
 [0.5970469 ]
 [0.41980717]
 [0.41869205]
 [0.33354294]
 [0.42821723]
 [0.31684992]
 [0.31134507]
 [0.51629543]
 [0.7274901 ]
 [0.4888919 ]
 [0.30533287]
 [0.3598683 ]
 [0.5187535 ]
 [0.5227133 ]
 [0.33146742]
 [0.43449536]
 [0.75575966]
 [0.56815463]
 [0.25779963]
 [0.47850165]
 [0.5355524 ]
 [0.34294078]
 [0.2103615 ]
 [0.27098617]
 [0.40102184]
 [0.20136704]
 [0.5042058 ]
 [0.4522678 ]
 [0.30499607]
 [0.38047168]
 [0.52894354]
 [0.7314439 ]
 [0.24363934]
 [0.3136542 ]
 [0.7726057 ]
 [0.32597068]
 [0.79271495]
 [0.33648944]
 [0.6768906 ]
 [0.51617604]
 [0.36306137]
 [0.16947944]
 [0.27388412]
 [0.61318487]
 [0.34958225]
 [0.48793668]
 [0.16796608]
 [0.28151014]
 [0.3553849 ]
 [0.43116567]
 [0.28812405]
 [0.42114505]]


In [19]:
len(output_players_performance)

54

In [20]:
filtered_sorted_batting_powerplay_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance
0,S Dhawan,3157,2584,86,36.709302,122.174923,0.505364
1,DA Warner,2892,2106,65,44.492308,137.321937,0.723681
2,CH Gayle,2373,1761,64,37.078125,134.752981,0.586194
3,G Gambhir,2277,1868,69,33.000000,121.895075,0.421668
4,V Kohli,2125,1825,58,36.637931,116.438356,0.417460
5,AM Rahane,2049,1783,65,31.523077,114.918676,0.335346
6,RV Uthappa,2035,1636,62,32.822581,124.388753,0.431178
7,PA Patel,1904,1565,73,26.082192,121.661342,0.315725
8,RG Sharma,1632,1389,55,29.672727,117.494600,0.317208
9,Q de Kock,1631,1248,45,36.244444,130.689103,0.513329


# Best Batters In the Powerplay

In [21]:
import pandas as pd

# Create a new dataframe from the array
performance_df = pd.DataFrame(output_players_performance, columns=['Predicted Performance'])

# Concatenate the two dataframes along the column axis
merged_batting_powerplay_df = pd.concat([filtered_sorted_batting_powerplay_df, performance_df], axis=1)
merged_batting_powerplay_df = merged_batting_powerplay_df.sort_values('Predicted Performance', ascending=False)
merged_batting_powerplay_df = merged_batting_powerplay_df.reset_index(drop=True)
merged_batting_powerplay_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Predicted Performance
0,JM Bairstow,695,470,14,49.642857,147.872340,0.796487,0.792715
1,SA Yadav,727,516,14,51.928571,140.891473,0.768834,0.772606
2,JC Buttler,1229,854,26,47.269231,143.911007,0.752485,0.755760
3,CA Lynn,779,540,17,45.823529,144.259259,0.719674,0.731444
4,DA Warner,2892,2106,65,44.492308,137.321937,0.723681,0.728027
5,KL Rahul,1594,1274,28,56.928571,125.117739,0.734117,0.727490
6,Ishan Kishan,679,551,12,56.583333,123.230490,0.679355,0.676891
7,ML Hayden,599,461,13,46.076923,129.934924,0.595999,0.613185
8,CH Gayle,2373,1761,64,37.078125,134.752981,0.586194,0.597047
9,DR Smith,1213,976,27,44.925926,124.282787,0.556716,0.568155


# Batting Middle Overs Part 1

In [22]:
batting_middle_one = []
for i in unique_batters:
    batting_middle_one.append(batting_stats_overs(i, get_batting_stats(i), 6, 10))

C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: invalid value encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:14: RuntimeWarning: invalid value encountered in scalar divide
  strike_rate = (runs_score/balls_faced)*100
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  average = runs_score/dismissals


In [23]:
batting_stats_overs("AB de Villiers", get_batting_stats("AB de Villiers") , 6, 10)

['AB de Villiers', 1295, 1067, 34, 38.088235294117645, 121.36832239925025]

In [24]:
batting_middle_one

[['YBK Jaiswal', 134, 97, 6, 22.333333333333332, 138.1443298969072],
 ['JC Buttler', 658, 493, 14, 47.0, 133.46855983772818],
 ['SV Samson', 1209, 901, 34, 35.55882352941177, 134.18423973362928],
 ['D Padikkal', 334, 280, 8, 41.75, 119.28571428571428],
 ['SO Hetmyer', 49, 47, 3, 16.333333333333332, 104.25531914893618],
 ['R Ashwin', 33, 39, 1, 33.0, 84.61538461538461],
 ['R Parag', 71, 78, 2, 35.5, 91.02564102564102],
 ['TA Boult', 0, 0, 0, nan, nan],
 ['OC McCoy', 0, 0, 0, nan, nan],
 ['WP Saha', 428, 460, 20, 21.4, 93.04347826086956],
 ['Shubman Gill', 479, 401, 17, 28.176470588235293, 119.45137157107233],
 ['MS Wade', 62, 64, 4, 15.5, 96.875],
 ['HH Pandya', 307, 250, 9, 34.111111111111114, 122.8],
 ['DA Miller', 490, 489, 10, 49.0, 100.20449897750512],
 ['V Kohli', 1801, 1583, 36, 50.02777777777778, 113.77132027795325],
 ['F du Plessis', 817, 712, 21, 38.904761904761905, 114.74719101123596],
 ['RM Patidar', 154, 100, 2, 77.0, 154.0],
 ['GJ Maxwell', 751, 546, 27, 27.814814814814813

In [25]:
batting_middle_one_df = pd.DataFrame(batting_middle_one, columns=['Batter', 'runs', 'balls', 'Dismissals', 'average', 'strike_rate'])
batting_middle_one_df

,Batter,runs,balls,Dismissals,average,strike_rate
0,YBK Jaiswal,134,97,6,22.333333,138.144330
1,JC Buttler,658,493,14,47.000000,133.468560
2,SV Samson,1209,901,34,35.558824,134.184240
3,D Padikkal,334,280,8,41.750000,119.285714
4,SO Hetmyer,49,47,3,16.333333,104.255319
...,...,...,...,...,...,...
600,D Kalyankrishna,0,0,0,NaN,NaN
601,MA Khote,0,0,0,NaN,NaN
602,SB Joshi,0,0,0,NaN,NaN
603,DS Lehmann,14,13,1,14.000000,107.692308


In [26]:
sorted_batting_middle_one_df = batting_middle_one_df.sort_values('runs', ascending=False)
filtered_sorted_batting_middle_one_df = sorted_batting_middle_one_df.query('runs >= 500').dropna()
print(len(filtered_sorted_batting_middle_one_df))
filtered_sorted_batting_middle_one_df

45


,Batter,runs,balls,Dismissals,average,strike_rate
14,V Kohli,1801,1583,36,50.027778,113.771320
179,SK Raina,1786,1460,48,37.208333,122.328767
43,S Dhawan,1624,1351,43,37.767442,120.207254
50,DA Warner,1546,1163,33,46.848485,132.932072
105,RV Uthappa,1462,1214,55,26.581818,120.428336
59,RG Sharma,1420,1336,54,26.296296,106.287425
176,AB de Villiers,1295,1067,34,38.088235,121.368322
189,CH Gayle,1221,859,31,39.387097,142.142026
2,SV Samson,1209,901,34,35.558824,134.184240
220,SR Watson,1158,800,36,32.166667,144.750000


In [27]:
# Reset index of filtered_sorted_batting_powerplay_df
filtered_sorted_batting_middle_one_df = filtered_sorted_batting_middle_one_df.reset_index(drop=True)
len(filtered_sorted_batting_middle_one_df)

45

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(filtered_sorted_batting_middle_one_df[['runs', 'average', 'strike_rate']])

# Define weights for the composite score
runs_weight = 0.1
avg_weight = 0.5
strike_rate_weight = 0.6

# Create a new column for the composite score
filtered_sorted_batting_middle_one_df['performance'] = (
    runs_weight * normalized_data[:, 0] +
    avg_weight * normalized_data[:, 1] +
    strike_rate_weight * normalized_data[:, 2]
)

# Split the data into features and target variable
X = normalized_data
y = filtered_sorted_batting_middle_one_df["performance"]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
2/2 [==============================] - 1s 3ms/step - loss: 0.6952 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6925 - accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6918 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6912 - accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6906 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6900 - accuracy: 0.0000e+00
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6896 - accuracy: 0.0000e+00
Epoch 10/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6890 - accuracy: 

In [29]:
predictions

array([[0.69516623],
       [0.6350142 ],
       [0.53659356],
       [0.59866583],
       [0.33432397],
       [0.56495166],
       [0.501084  ],
       [0.24244858],
       [0.265443  ]], dtype=float32)

In [30]:
len(predictions)

9

In [31]:
output_players_performance = model.predict(X)
print(output_players_performance)

2/2 [==============================] - 0s 2ms/step
[[0.6441382 ]
 [0.56495166]
 [0.542152  ]
 [0.736604  ]
 [0.42027923]
 [0.33432397]
 [0.53659356]
 [0.7019715 ]
 [0.59866583]
 [0.6350142 ]
 [0.41387528]
 [0.38820368]
 [0.2959589 ]
 [0.41843286]
 [0.5711875 ]
 [0.5086552 ]
 [0.3574361 ]
 [0.846193  ]
 [0.871443  ]
 [0.44245815]
 [0.41791523]
 [0.5407837 ]
 [0.5374307 ]
 [0.5021327 ]
 [0.7838889 ]
 [0.33649606]
 [0.2636683 ]
 [0.23297168]
 [0.338989  ]
 [0.46070576]
 [0.69516623]
 [0.40863922]
 [0.4474988 ]
 [0.26041588]
 [0.5464613 ]
 [0.34473425]
 [0.37258434]
 [0.24244855]
 [0.21299846]
 [0.265443  ]
 [0.55922735]
 [0.4893733 ]
 [0.5377305 ]
 [0.56340367]
 [0.501084  ]]


In [32]:
len(output_players_performance)

45

In [33]:
filtered_sorted_batting_middle_one_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance
0,V Kohli,1801,1583,36,50.027778,113.771320,0.648259
1,SK Raina,1786,1460,48,37.208333,122.328767,0.563766
2,S Dhawan,1624,1351,43,37.767442,120.207254,0.541046
3,DA Warner,1546,1163,33,46.848485,132.932072,0.743048
4,RV Uthappa,1462,1214,55,26.581818,120.428336,0.398381
5,RG Sharma,1420,1336,54,26.296296,106.287425,0.279636
6,AB de Villiers,1295,1067,34,38.088235,121.368322,0.528671
7,CH Gayle,1221,859,31,39.387097,142.142026,0.703023
8,SV Samson,1209,901,34,35.558824,134.184240,0.593839
9,SR Watson,1158,800,36,32.166667,144.750000,0.633686


# Best Batters In the Middle Part 1

In [34]:
import pandas as pd

# Create a new dataframe from the array
performance_df = pd.DataFrame(output_players_performance, columns=['Actual Performance'])

# Concatenate the two dataframes along the column axis
merged_filtered_sorted_batting_middle_one_df = pd.concat([filtered_sorted_batting_middle_one_df, performance_df], axis=1)
merged_filtered_sorted_batting_middle_one_df = merged_filtered_sorted_batting_middle_one_df.sort_values('Actual Performance', ascending=False)
merged_filtered_sorted_batting_middle_one_df = merged_filtered_sorted_batting_middle_one_df.reset_index(drop=True)
merged_filtered_sorted_batting_middle_one_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Actual Performance
0,SE Marsh,856,604,14,61.142857,141.721854,0.928146,0.871443
1,RR Pant,913,678,15,60.866667,134.660767,0.873288,0.846193
2,V Sehwag,749,449,20,37.450000,166.815145,0.839446,0.783889
3,DA Warner,1546,1163,33,46.848485,132.932072,0.743048,0.736604
4,CH Gayle,1221,859,31,39.387097,142.142026,0.703023,0.701971
5,JC Buttler,658,493,14,47.000000,133.468560,0.680623,0.695166
6,V Kohli,1801,1583,36,50.027778,113.771320,0.648259,0.644138
7,SR Watson,1158,800,36,32.166667,144.750000,0.633686,0.635014
8,SV Samson,1209,901,34,35.558824,134.184240,0.593839,0.598666
9,KL Rahul,989,781,25,39.560000,126.632522,0.564183,0.571187


# Batting Middle Overs Part 2

In [35]:
batting_middle_two = []
for i in unique_batters:
    batting_middle_two.append(batting_stats_overs(i, get_batting_stats(i), 11, 15))

C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: invalid value encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:14: RuntimeWarning: invalid value encountered in scalar divide
  strike_rate = (runs_score/balls_faced)*100
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  average = runs_score/dismissals


In [36]:
batting_middle_two

[['YBK Jaiswal', 47, 37, 5, 9.4, 127.02702702702702],
 ['JC Buttler', 616, 381, 19, 32.421052631578945, 161.67979002624674],
 ['SV Samson', 880, 621, 29, 30.344827586206897, 141.70692431561997],
 ['D Padikkal', 265, 193, 11, 24.09090909090909, 137.30569948186528],
 ['SO Hetmyer', 298, 232, 8, 37.25, 128.44827586206898],
 ['R Ashwin', 175, 167, 9, 19.444444444444443, 104.79041916167664],
 ['R Parag', 196, 165, 7, 28.0, 118.7878787878788],
 ['TA Boult', 2, 6, 1, 2.0, 33.33333333333333],
 ['OC McCoy', 0, 0, 0, nan, nan],
 ['WP Saha', 586, 430, 17, 34.470588235294116, 136.27906976744185],
 ['Shubman Gill', 389, 273, 10, 38.9, 142.49084249084248],
 ['MS Wade', 4, 9, 3, 1.3333333333333333, 44.44444444444444],
 ['HH Pandya', 574, 442, 17, 33.76470588235294, 129.86425339366517],
 ['DA Miller', 1083, 762, 26, 41.65384615384615, 142.1259842519685],
 ['V Kohli', 1707, 1230, 51, 33.470588235294116, 138.78048780487805],
 ['F du Plessis', 740, 555, 23, 32.17391304347826, 133.33333333333331],
 ['RM P

In [37]:
batting_middle_two_df = pd.DataFrame(batting_middle_two, columns=['Batter', 'runs', 'balls', 'Dismissals', 'average', 'strike_rate'])
batting_middle_two_df

,Batter,runs,balls,Dismissals,average,strike_rate
0,YBK Jaiswal,47,37,5,9.400000,127.027027
1,JC Buttler,616,381,19,32.421053,161.679790
2,SV Samson,880,621,29,30.344828,141.706924
3,D Padikkal,265,193,11,24.090909,137.305699
4,SO Hetmyer,298,232,8,37.250000,128.448276
...,...,...,...,...,...,...
600,D Kalyankrishna,0,0,0,NaN,NaN
601,MA Khote,12,9,1,12.000000,133.333333
602,SB Joshi,6,14,2,3.000000,42.857143
603,DS Lehmann,0,2,1,0.000000,0.000000


In [38]:
sorted_batting_middle_two_df = batting_middle_two_df.sort_values('runs', ascending=False)
filtered_sorted_batting_middle_two_df = sorted_batting_middle_two_df.query('runs >= 500').dropna()
print(len(filtered_sorted_batting_middle_two_df))
filtered_sorted_batting_middle_two_df

43


,Batter,runs,balls,Dismissals,average,strike_rate
70,MS Dhoni,1781,1591,39,45.666667,111.942175
176,AB de Villiers,1724,1119,37,46.594595,154.066130
14,V Kohli,1707,1230,51,33.470588,138.780488
59,RG Sharma,1682,1220,37,45.459459,137.868852
19,KD Karthik,1590,1188,53,30.000000,133.838384
179,SK Raina,1526,1026,54,28.259259,148.732943
69,AT Rayudu,1467,1047,48,30.562500,140.114613
116,KA Pollard,1366,1033,34,40.176471,132.236205
262,Yuvraj Singh,1278,931,43,29.720930,137.271751
241,YK Pathan,1206,847,44,27.409091,142.384888


In [39]:
# Reset index of filtered_sorted_batting_powerplay_df
filtered_sorted_batting_middle_two_df = filtered_sorted_batting_middle_two_df.reset_index(drop=True)
len(filtered_sorted_batting_middle_two_df)

43

In [40]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(filtered_sorted_batting_middle_two_df[['runs', 'average', 'strike_rate']])

# Define weights for the composite score
runs_weight = 0.1
avg_weight = 0.5
strike_rate_weight = 0.7

# Create a new column for the composite score
filtered_sorted_batting_middle_two_df['performance'] = (
    runs_weight * normalized_data[:, 0] +
    avg_weight * normalized_data[:, 1] +
    strike_rate_weight * normalized_data[:, 2]
)

# Split the data into features and target variable
X = normalized_data
y = filtered_sorted_batting_middle_two_df["performance"]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
2/2 [==============================] - 1s 4ms/step - loss: 0.6908 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6896 - accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6887 - accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6883 - accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6881 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6877 - accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6873 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6869 - accuracy: 0.0000e+00
Epoch 9/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6865 - accuracy: 0.0000e+00
Epoch 10/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6861 - accuracy: 

In [41]:
predictions

array([[0.57701   ],
       [0.60795426],
       [0.28661063],
       [0.71147984],
       [0.46175086],
       [0.3229085 ],
       [0.41364053],
       [0.4336536 ],
       [0.6157041 ]], dtype=float32)

In [42]:
len(predictions)

9

In [43]:
output_players_performance = model.predict(X)
print(output_players_performance)

2/2 [==============================] - 0s 2ms/step
[[0.40175822]
 [0.7421847 ]
 [0.49271223]
 [0.6119962 ]
 [0.41134942]
 [0.5040595 ]
 [0.46175086]
 [0.4971264 ]
 [0.41964012]
 [0.42623067]
 [0.5697215 ]
 [0.57790357]
 [0.4163043 ]
 [0.4937924 ]
 [0.3750894 ]
 [0.665859  ]
 [0.8746085 ]
 [0.28661063]
 [0.3229085 ]
 [0.60795426]
 [0.43190378]
 [0.6157041 ]
 [0.39163643]
 [0.71147984]
 [0.42512986]
 [0.3796195 ]
 [0.58947164]
 [0.4336536 ]
 [0.45302156]
 [0.32083797]
 [0.2931368 ]
 [0.57701   ]
 [0.5591504 ]
 [0.5747863 ]
 [0.41364053]
 [0.30655718]
 [0.3593187 ]
 [0.27424464]
 [0.26577947]
 [0.4620901 ]
 [0.29686028]
 [0.2648662 ]
 [0.31482062]]


In [44]:
len(output_players_performance)

43

In [45]:
filtered_sorted_batting_middle_two_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance
0,MS Dhoni,1781,1591,39,45.666667,111.942175,0.399611
1,AB de Villiers,1724,1119,37,46.594595,154.066130,0.798610
2,V Kohli,1707,1230,51,33.470588,138.780488,0.501598
3,RG Sharma,1682,1220,37,45.459459,137.868852,0.631086
4,KD Karthik,1590,1188,53,30.000000,133.838384,0.405836
5,SK Raina,1526,1026,54,28.259259,148.732943,0.519326
6,AT Rayudu,1467,1047,48,30.562500,140.114613,0.461249
7,KA Pollard,1366,1033,34,40.176471,132.236205,0.492111
8,Yuvraj Singh,1278,931,43,29.720930,137.271751,0.410091
9,YK Pathan,1206,847,44,27.409091,142.384888,0.425114


# Best Batters In the Middle Part 2

In [46]:
import pandas as pd

# Create a new dataframe from the array
performance_df = pd.DataFrame(output_players_performance, columns=['Actual Performance'])

# Concatenate the two dataframes along the column axis
merged_filtered_sorted_batting_middle_two_df = pd.concat([filtered_sorted_batting_middle_two_df, performance_df], axis=1)
merged_filtered_sorted_batting_middle_two_df = merged_filtered_sorted_batting_middle_two_df.sort_values('Actual Performance', ascending=False)
merged_filtered_sorted_batting_middle_two_df = merged_filtered_sorted_batting_middle_two_df.reset_index(drop=True)
merged_filtered_sorted_batting_middle_two_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Actual Performance
0,CH Gayle,967,517,18,53.722222,187.040619,1.129751,0.874609
1,AB de Villiers,1724,1119,37,46.594595,154.066130,0.798610,0.742185
2,KL Rahul,754,517,12,62.833333,145.841393,0.835366,0.711480
3,SR Watson,1006,593,31,32.451613,169.645868,0.722379,0.665859
4,GJ Maxwell,848,492,30,28.266667,172.357724,0.686403,0.615704
5,RG Sharma,1682,1220,37,45.459459,137.868852,0.631086,0.611996
6,AD Russell,893,559,26,34.346154,159.749553,0.643380,0.607954
7,KS Williamson,730,478,19,38.421053,152.719665,0.612627,0.589472
8,DA Warner,1044,699,28,37.285714,149.356223,0.592670,0.577904
9,SE Marsh,684,427,21,32.571429,160.187354,0.610340,0.577010


# Batting Death Overs

In [47]:
batting_death = []
for i in unique_batters:
    batting_death.append(batting_stats_overs(i, get_batting_stats(i), 16, 19))

C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: invalid value encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:14: RuntimeWarning: invalid value encountered in scalar divide
  strike_rate = (runs_score/balls_faced)*100
C:\Users\anubr\AppData\Local\Temp\ipykernel_30576\4238514807.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  average = runs_score/dismissals


In [48]:
batting_death

[['YBK Jaiswal', 0, 0, 0, nan, nan],
 ['JC Buttler', 328, 163, 12, 27.333333333333332, 201.22699386503066],
 ['SV Samson', 438, 227, 23, 19.043478260869566, 192.95154185022025],
 ['D Padikkal', 30, 28, 7, 4.285714285714286, 107.14285714285714],
 ['SO Hetmyer', 431, 216, 14, 30.785714285714285, 199.53703703703704],
 ['R Ashwin', 381, 288, 34, 11.205882352941176, 132.29166666666669],
 ['R Parag', 255, 164, 21, 12.142857142857142, 155.4878048780488],
 ['TA Boult', 51, 42, 4, 12.75, 121.42857142857142],
 ['OC McCoy', 8, 5, 1, 8.0, 160.0],
 ['WP Saha', 441, 281, 24, 18.375, 156.93950177935943],
 ['Shubman Gill', 150, 114, 10, 15.0, 131.57894736842107],
 ['MS Wade', 0, 0, 0, nan, nan],
 ['HH Pandya', 998, 538, 36, 27.72222222222222, 185.5018587360595],
 ['DA Miller', 795, 439, 26, 30.576923076923077, 181.09339407744875],
 ['V Kohli', 991, 491, 38, 26.07894736842105, 201.8329938900204],
 ['F du Plessis', 366, 183, 17, 21.529411764705884, 200.0],
 ['RM Patidar', 20, 10, 0, inf, 200.0],
 ['GJ M

In [49]:
batting_death_df = pd.DataFrame(batting_death, columns=['Batter', 'runs', 'balls', 'Dismissals', 'average', 'strike_rate'])
batting_death_df

,Batter,runs,balls,Dismissals,average,strike_rate
0,YBK Jaiswal,0,0,0,NaN,NaN
1,JC Buttler,328,163,12,27.333333,201.226994
2,SV Samson,438,227,23,19.043478,192.951542
3,D Padikkal,30,28,7,4.285714,107.142857
4,SO Hetmyer,431,216,14,30.785714,199.537037
...,...,...,...,...,...,...
600,D Kalyankrishna,3,8,1,3.000000,37.500000
601,MA Khote,12,13,1,12.000000,92.307692
602,SB Joshi,0,0,0,NaN,NaN
603,DS Lehmann,0,0,0,NaN,NaN


In [50]:
sorted_batting_death_df = batting_death_df.sort_values('runs', ascending=False)
filtered_sorted_batting_death_df = sorted_batting_death_df.query('runs >= 300').dropna()
print(len(filtered_sorted_batting_death_df))
filtered_sorted_batting_death_df

58


,Batter,runs,balls,Dismissals,average,strike_rate
70,MS Dhoni,2530,1352,73,34.657534,187.130178
116,KA Pollard,1708,941,69,24.753623,181.509033
176,AB de Villiers,1421,611,37,38.405405,232.569558
19,KD Karthik,1282,695,54,23.740741,184.460432
135,RA Jadeja,1155,730,52,22.211538,158.219178
59,RG Sharma,1145,581,48,23.854167,197.074010
12,HH Pandya,998,538,36,27.722222,185.501859
14,V Kohli,991,491,38,26.078947,201.832994
241,YK Pathan,858,515,33,26.000000,166.601942
80,AD Russell,854,408,36,23.722222,209.313725


In [51]:
# Reset index of filtered_sorted_batting_powerplay_df
filtered_sorted_batting_death_df = filtered_sorted_batting_death_df.reset_index(drop=True)
len(filtered_sorted_batting_death_df)

58

In [52]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(filtered_sorted_batting_death_df[['runs', 'average', 'strike_rate']])

# Define weights for the composite score
runs_weight = 0.1
avg_weight = 0.3
strike_rate_weight = 0.7

# Create a new column for the composite score
filtered_sorted_batting_death_df['performance'] = (
    runs_weight * normalized_data[:, 0] +
    avg_weight * normalized_data[:, 1] +
    strike_rate_weight * normalized_data[:, 2]
)

# Split the data into features and target variable
X = normalized_data
y = filtered_sorted_batting_death_df["performance"]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
2/2 [==============================] - 1s 3ms/step - loss: 0.6937 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6917 - accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6906 - accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6894 - accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6885 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6873 - accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6854 - accuracy: 0.0000e+00
Epoch 9/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6844 - accuracy: 0.0000e+00
Epoch 10/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6831 - accuracy: 

In [53]:
predictions

array([[0.5957521 ],
       [0.45834494],
       [0.3014401 ],
       [0.4629244 ],
       [0.5532884 ],
       [0.9138559 ],
       [0.61168176],
       [0.5997617 ],
       [0.304315  ],
       [0.4831926 ],
       [0.58640945],
       [0.4868787 ]], dtype=float32)

In [54]:
len(predictions)

12

In [55]:
output_players_performance = model.predict(X)
print(output_players_performance)

2/2 [==============================] - 0s 3ms/step
[[0.74775946]
 [0.58640945]
 [0.9138559 ]
 [0.5689196 ]
 [0.35974628]
 [0.649137  ]
 [0.6055995 ]
 [0.6969205 ]
 [0.44344476]
 [0.714725  ]
 [0.37050617]
 [0.5706089 ]
 [0.5997617 ]
 [0.5957521 ]
 [0.48570484]
 [0.42165297]
 [0.24796432]
 [0.3586124 ]
 [0.33641785]
 [0.46855998]
 [0.23969796]
 [0.7234621 ]
 [0.62820876]
 [0.29445547]
 [0.27046832]
 [0.45834494]
 [0.4736949 ]
 [0.2452998 ]
 [0.4868787 ]
 [0.304315  ]
 [0.27721933]
 [0.528308  ]
 [0.5532884 ]
 [0.709739  ]
 [0.4831926 ]
 [0.4629244 ]
 [0.39732945]
 [0.77122056]
 [0.34526563]
 [0.61168176]
 [0.33810675]
 [0.13981049]
 [0.29628703]
 [0.14038885]
 [0.4281861 ]
 [0.606045  ]
 [0.34146544]
 [0.4054333 ]
 [0.3014401 ]
 [0.32266954]
 [0.6796672 ]
 [0.2806276 ]
 [0.23467383]
 [0.16398615]
 [0.5492843 ]
 [0.23293252]
 [0.6084922 ]
 [0.18434846]]


In [56]:
len(output_players_performance)

58

In [57]:
filtered_sorted_batting_death_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance
0,MS Dhoni,2530,1352,73,34.657534,187.130178,0.749985
1,KA Pollard,1708,941,69,24.753623,181.509033,0.574967
2,AB de Villiers,1421,611,37,38.405405,232.569558,1.036712
3,KD Karthik,1282,695,54,23.740741,184.460432,0.564861
4,RA Jadeja,1155,730,52,22.211538,158.219178,0.371324
5,RG Sharma,1145,581,48,23.854167,197.074010,0.642647
6,HH Pandya,998,538,36,27.722222,185.501859,0.599647
7,V Kohli,991,491,38,26.078947,201.832994,0.689707
8,YK Pathan,858,515,33,26.000000,166.601942,0.451730
9,AD Russell,854,408,36,23.722222,209.313725,0.708560


# Best Batters In the Death

In [58]:
import pandas as pd

# Create a new dataframe from the array
performance_df = pd.DataFrame(output_players_performance, columns=['Actual Performance'])

# Concatenate the two dataframes along the column axis
merged_filtered_sorted_batting_death_df = pd.concat([filtered_sorted_batting_death_df, performance_df], axis=1)
merged_filtered_sorted_batting_death_df = merged_filtered_sorted_batting_death_df.sort_values('Actual Performance', ascending=False)
merged_filtered_sorted_batting_death_df = merged_filtered_sorted_batting_death_df.reset_index(drop=True)
merged_filtered_sorted_batting_death_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Actual Performance
0,AB de Villiers,1421,611,37,38.405405,232.569558,1.036712,0.913856
1,CH Gayle,404,196,12,33.666667,206.122449,0.769070,0.771221
2,MS Dhoni,2530,1352,73,34.657534,187.130178,0.749985,0.747759
3,KL Rahul,552,283,16,34.500000,195.053004,0.711589,0.723462
4,AD Russell,854,408,36,23.722222,209.313725,0.708560,0.714725
5,SO Hetmyer,431,216,14,30.785714,199.537037,0.697617,0.709739
6,V Kohli,991,491,38,26.078947,201.832994,0.689707,0.696921
7,JC Buttler,328,163,12,27.333333,201.226994,0.668795,0.679667
8,RG Sharma,1145,581,48,23.854167,197.074010,0.642647,0.649137
9,RR Pant,534,264,25,21.360000,202.272727,0.623845,0.628209
